In [ ]:
# grp_opts.py
# group options based on right, symbol and strike.
# this makes it easy to delete unwanted ones on inspection.
import pandas as pd

def grp_opts(df):
    '''Groups options and sorts strikes by puts and calls
    Arg: 
       df as dataframe. Requires 'symbol', 'strike' and 'dte' fields in the df
    Returns: sorted dataframe'''
    
    gb = df.groupby('right')

    if 'C' in [k for k in gb.indices]:
        df_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'dte', 'strike'], ascending=[True, False, True])
    else:
        df_calls =  pd.DataFrame([])

    if 'P' in [k for k in gb.indices]:
        df_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'dte', 'strike'], ascending=[True, False, False])
    else:
        df_puts =  pd.DataFrame([])

    df = pd.concat([df_puts, df_calls]).reset_index(drop=True)
    
    return df

In [ ]:
# get_prec.py
# get precision, based on the base
from math import floor, log10

def get_prec(v, base):
    '''gives the precision value
    args:
       (v) as value needing precision in float
       (base) as the base value e.g. 0.05'''
    
    return round(round((v)/ base) * base, -int(floor(log10(base))))

In [1]:
from ib_insync import *
util.startLoop()
from helper import get_snp_remqty, get_p_remqty, get_snps, get_prec
with IB().connect('127.0.0.1', 1300, clientId=2) as ib:
    remqty = get_snp_remqty(ib=ib, remqty_p=get_p_remqty(ib), undContracts=get_snps(ib))

Started to throttle requests
Stopped to throttle requests


In [106]:
from os import listdir
import pandas as pd
import sys

sys.path.append('./py')

from helper import grp_opts, get_prec

fspath = '../data/snp/' # path for pickles

minOptPrice = 0.1
minRom = 0.85

fs = listdir(fspath)

cols = ['optId', 'symbol', 'right', 'expiration', 'dte', 'strike', 'undPrice', 
'lo52', 'hi52', 'Fall', 'Rise', 'loFall', 'hiRise', 'std3', 'loStd3', 'hiStd3', 
'lotsize', 'optPrice', 'optMargin', 'rom']

optsList = [f for f in fs if f[-3:] == 'pkl']

df1 = pd.concat([pd.read_pickle(fspath+f) for f in optsList], axis=0, sort=True).reset_index(drop=True)[cols]

df2 = df1[((df1.strike > df1.hi52) | (df1.strike < df1.lo52)) & (df1.optPrice > minOptPrice) & (df1.rom > minRom)]

df2 = grp_opts(df2)

df2 = df2.assign(remqty=[remqty[u] for u in df2.symbol])

df3 = df2.groupby('symbol').apply(lambda x: x.nlargest(3, 'rom'))

df4 = df3.assign(expPrice=[get_prec(p*1.1, 0.05) for p in df3.optPrice])
df4 = df4.assign(expQty=(df4.remqty/3).astype('int'))

# high cost options
df_single = df4[df4.expQty == 0].set_index('optId').groupby('symbol').head(1)
df_single.loc[:, 'expQty'] = 1    # set quantity to 1

# remove high cost options
df_multiple = df4[df4.expQty != 0].set_index('optId')

# final set of options
df = pd.concat([df_single, df_multiple], axis=0).reset_index()

In [111]:
df

,optId,symbol,right,expiration,dte,strike,undPrice,lo52,hi52,Fall,...,std3,loStd3,hiStd3,lotsize,optPrice,optMargin,rom,remqty,expPrice,expQty
0,359772916,AMZN,C,20190426,9,2420.0,1867.875,1086.86,2050.50,397.99,...,163.159007,1704.715993,2031.034007,1,0.110,3.03,1.016502,1,0.10,1
1,356373262,BKNG,P,20190426,9,1600.0,1848.450,1606.27,2228.99,293.50,...,114.022438,1734.427562,1962.472438,1,2.325,30.11,2.162072,1,2.55,1
2,360779839,GOOG,C,20190426,9,1315.0,1239.880,970.11,1273.89,186.21,...,56.671037,1183.208963,1296.551037,1,0.225,4.67,1.349036,1,0.25,1
3,356144511,ABT,C,20190426,8,81.0,74.250,53.96,80.74,7.70,...,5.498927,68.751073,79.748927,1,0.110,0.54,6.416667,16,0.10,5
4,358199571,ABT,C,20190503,15,81.5,74.250,53.96,80.74,9.48,...,5.498927,68.751073,79.748927,1,0.160,3.00,0.896000,16,0.20,5
5,361113639,INTC,C,20190426,9,64.5,58.850,42.04,59.59,8.11,...,4.600641,54.249359,63.450641,1,0.130,2.04,1.784314,20,0.15,6
6,360781346,INTC,C,20190426,9,63.5,58.850,42.04,59.59,8.11,...,4.600641,54.249359,63.450641,1,0.210,3.39,1.734513,20,0.25,6
7,360781366,INTC,C,20190426,9,64.0,58.850,42.04,59.59,8.11,...,4.600641,54.249359,63.450641,1,0.160,2.79,1.605735,20,0.20,6
8,358277820,LLY,C,20190503,15,135.0,117.025,73.69,132.13,14.96,...,10.771897,106.253103,127.796897,1,0.130,1.95,1.120000,10,0.15,3
9,357128625,NFLX,C,20190426,9,485.0,361.860,178.38,423.21,104.60,...,21.854200,340.005800,383.714200,1,0.120,0.52,6.461538,3,0.15,1


In [109]:
with IB().connect('127.0.0.1', 1300, clientId=2) as ib:
    contracts = [Contract(conId=i) for i in df.optId]
    qcs = ib.qualifyContracts(*contracts)
#     tickers = ib.reqTickers(*qcs)
    orders = [LimitOrder(action='SELL', totalQuantity=qty, lmtPrice=tgtPrice) for qty, tgtPrice in zip(df.expQty, df.expPrice)]